In [1]:
from google.colab import drive
drive.mount('/content/drive/')
import os

os.chdir("/content/drive/My Drive/MIMIC/mca_bert/mc_ptm")
os.listdir("/content/drive/My Drive/MIMIC/mca_bert/mc_ptm")

Mounted at /content/drive/


['README.md',
 'utils',
 'models',
 'data',
 '__pycache__',
 'cache',
 'saved',
 'main.py',
 'train_test.py']

In [2]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
!python main.py

Streaming output truncated to the last 5000 lines.
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0

# Utils 

In [5]:
from utils.dataloader import load_lookups, prepare_instance_bert, MyDataset, my_collate_bert
from utils.options import args_parser
import torch
import random
import numpy as np

In [6]:
import sys
sys.argv = ['./utils/options.py']
args = args_parser()
args

Namespace(BERT_MAX_LENGTH=512, DATA_DIR='./data', MAX_LENGTH=500, MIMIC_2_DIR='./data/mimic2', MIMIC_3_DIR='./data/mimic3', MODEL_DIR='./saved/models', Y='50', batch_size_test=8, batch_size_train=16, cache_dir='./cache/torch/transformers', chunk_size=512, command='python ./utils/options.py', criterion='f1_macro', data_path='./data/mimic3/train_50.csv', debug=0, decoder='lan', drop_samples=None, dropout=0.2, gpu=0, job_id=None, lower_cased=False, lr=0.0001, lr_layer_decay=1.0, model='hibert', n_epochs=1, n_heads=2, num_workers=1, optimizer='Adam', patience=10, pretrained_model='emilyalsentzer/Bio_ClinicalBERT', print_every=50, random_seed=1, test_model=None, transformer_encoder_num_layers=2, use_lr_layer_decay=False, use_lr_scheduler=False, verbose=1, version='mimic3', vocab='./data/mimic3/vocab.csv', warm_up=0.1, weight_decay=0)

In [7]:
dicts = load_lookups(args)
#print(dicts)

In [8]:
print('now start prepare instance bert')
#train_instances = prepare_instance_bert(dicts,args.data_path, args, args.MAX_LENGTH)
print("train_instances {}".format(len(train_instances)))

now start prepare instance bert


NameError: ignored

In [ ]:
print('now start prepare test instance bert')
#test_instances = prepare_instance_bert(dicts,args.data_path.replace('train', 'test'), args, args.MAX_LENGTH)
print("test_instances {}".format(len(test_instances)))

In [ ]:
print('now start prepare dev instance bert')
#dev_instances = prepare_instance_bert(dicts,args.data_path.replace('train', 'dev'), args, args.MAX_LENGTH)
print("test_instances {}".format(len(dev_instances)))

In [ ]:
train_instances[0]

In [ ]:
import json
import numpy as np

pro_dev_instances = []
for single_instance in dev_instances:
  label = [] # float
  tokens= single_instance['tokens']
  tokens_id = [] # int
  segments = [] #int 
  masks = [] #int
  hadm_id = single_instance['hadm_id']
  
  for single_label in single_instance['label']:
    label.append(str(single_label))
  for single_tokens_id in single_instance['tokens_id']:
    tokens_id.append(str(single_tokens_id))
  for single_segment in single_instance['segments']:
    segments.append(str(single_segment))
  for single_masks in single_instance['masks']:
    masks.append(str(single_masks))
  pro_dev_instances.append({'label':label,
                              'tokens':tokens,
                              'tokens_id':tokens_id,
                              'segments':segments,
                              'masks':masks,
                              'hadm_id':hadm_id})
# pro_train_instances[0]
with open('./data/mimic3/dev_instances.txt', 'w') as f:
    f.write(json.dumps(pro_dev_instances))

In [ ]:
#Now read the file back into a Python list object
with open('./data/mimic3/train_instances.txt', 'r') as f:
    a = json.loads(f.read())
    train_instances = []    
for single_instance in a:
  label = [] # float
  tokens= single_instance['tokens']
  tokens_id = [] # int
  segments = [] #int 
  masks = [] #int
  hadm_id = single_instance['hadm_id']
  
  for single_label in single_instance['label']:
    label.append(float(single_label))
  for single_tokens_id in single_instance['tokens_id']:
    tokens_id.append(int(single_tokens_id))
  for single_segment in single_instance['segments']:
    segments.append(int(single_segment))
  for single_masks in single_instance['masks']:
    masks.append(int(single_masks))
  train_instances.append({'label':np.array(label),
                          'tokens':tokens,
                          'tokens_id':np.array(tokens_id),
                          'segments':np.array(segments),
                          'masks':np.array(masks),
                          'hadm_id':hadm_id})

In [ ]:
train_instances[0]

In [ ]:
args.data_path.replace('train', 'test')

In [9]:
a = torch.randn(2,3,4)
a


tensor([[[ 3.9230e-01,  2.3209e+00, -3.9858e-01, -1.4068e+00],
         [ 1.0050e+00, -4.0137e-01, -1.0806e+00,  4.9223e-02],
         [ 3.7113e-01,  1.0511e+00, -1.4470e-01,  2.5431e-01]],

        [[-1.6305e-03,  3.1508e-01,  1.7935e+00, -9.3272e-01],
         [-1.5809e+00,  1.2995e+00, -2.5176e+00, -1.0225e-01],
         [ 6.9152e-01,  1.5785e+00,  1.7198e+00, -3.3375e-02]]])

In [10]:
a.max(dim=1)

torch.return_types.max(
values=tensor([[ 1.0050,  2.3209, -0.1447,  0.2543],
        [ 0.6915,  1.5785,  1.7935, -0.0334]]),
indices=tensor([[1, 0, 2, 2],
        [2, 2, 0, 2]]))

In [11]:
a.max(dim=1).shape

AttributeError: ignored